In [1]:
# Analyais on Fan 对番种之研究
import numpy as np
import os
from collections import defaultdict
import rule
def default_zero():
    return 0
def default_list():
    return []

In [2]:
# Total Tile Combinations 所有可能手牌组合 C136, 14
combination = 1
for i in range(14):
    combination*=(136-i)/(14-i)
print("总共有{:.3e}种组合数。".format(combination))

总共有4.250e+18种组合数。


In [3]:
tile_list_raw = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9',  #饼
            'W1', 'W2', 'W3', 'W4', 'W5', 'W6', 'W7', 'W8', 'W9',   #万
            'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9',   #条
            'F1', 'F2', 'F3', 'F4', 'J1', 'J2', 'J3' #风、箭
            ]

In [4]:
triplet_collection = []
duo_collection = []
for i in range(len(tile_list_raw)):
    tile = tile_list_raw[i]
    tile_type = tile[0]
    rank = tile[1]
    tile_next = None
    tile_prev = None
    if i!=0 :
        tile_prev = tile_list_raw[i-1]
    if i!= len(tile_list_raw)-1:
        tile_next = tile_list_raw[i+1]
    # add duo
    duo_collection.append({tile:2})
    # add trio
    triplet_collection.append({tile:3})
    # add straight
    if tile_next and tile_prev  and tile_next[0]==tile_type and tile_prev[0]==tile_type and tile_type in ["B","W","T"]:
        triplet_collection.append({tile_prev:1, tile:1, tile_next:1})


In [5]:
def test_validity(list_of_combination_dicts):
    """
    Test if any tile_comb exceed limit of 4
    """
    tile_holder = defaultdict(default_zero)
    for combination_dict in list_of_combination_dicts:
        for k in combination_dict:
            tile_holder[k]+=combination_dict[k]
            if tile_holder[k]>4:
                return False
    return True


In [6]:
# Brute-force through all possible combinations and discard the unfit
def brute_force_filter(first_triplet, triplet_collection, duo_collection):
    """
    Fixed first triplet for parallel processing
    """
    valid_generic_form = 0
    preset_multiplier = [4,6,4,1]
    for second_triplet in triplet_collection:
        if test_validity([first_triplet, second_triplet]):
            for third_triplet in triplet_collection:
                if test_validity([first_triplet, second_triplet, third_triplet]):
                    for fourth_triplet in triplet_collection:
                        if test_validity([first_triplet, second_triplet, third_triplet, fourth_triplet]):
                            for duo in duo_collection:
                                if test_validity([first_triplet, second_triplet, third_triplet, fourth_triplet, duo]):
                                    tmp_hand = defaultdict(default_zero)
                                    # calculate possibilities of forming such hand
                                    formation_multiplier = 1
                                    win_tile = None
                                    for k in first_triplet:
                                        tmp_hand[k]+=first_triplet[k]
                                    for k in second_triplet:
                                        tmp_hand[k]+=second_triplet[k]
                                    for k in third_triplet:
                                        tmp_hand[k]+=third_triplet[k]
                                    for k in fourth_triplet:
                                        tmp_hand[k]+=fourth_triplet[k]
                                    for k in duo:
                                        tmp_hand[k]+=duo[k]
                                        win_tile = k
                                    for k in tmp_hand:
                                        formation_multiplier*=preset_multiplier[tmp_hand[k]-1]
                                    valid_generic_form+=formation_multiplier
    return valid_generic_form

    

In [7]:
# Brute-force through all possible combinations
def brute_force_analysis(triplet_collection, duo_collection):
    """
    Fixed first triplet for parallel processing
    """
    preset_multiplier = [4,6,4,1]
    cumulative_fan = defaultdict(default_zero)
    for first_triplet in triplet_collection:
        for second_triplet in triplet_collection:
            for third_triplet in triplet_collection:
                for fourth_triplet in triplet_collection:
                    for duo in duo_collection:
                        if test_validity([first_triplet, second_triplet, third_triplet, fourth_triplet, duo]):
                            tmp_hand = defaultdict(default_zero)
                            # calculate possibilities of forming such hand
                            formation_multiplier = 1
                            win_tile = None
                            for k in first_triplet:
                                tmp_hand[k]+=first_triplet[k]
                            for k in second_triplet:
                                tmp_hand[k]+=second_triplet[k]
                            for k in third_triplet:
                                tmp_hand[k]+=third_triplet[k]
                            for k in fourth_triplet:
                                tmp_hand[k]+=fourth_triplet[k]
                            for k in duo:
                                tmp_hand[k]+=duo[k]
                                win_tile = k
                            for k in tmp_hand:
                                formation_multiplier*=preset_multiplier[tmp_hand[k]-1]
                            fan_sum, fan_list = rule.calc_exact_fan_with_PyMahJongGB([], tmp_hand, win_tile, False,False, False, False, 1,0)
                            if fan_sum>=8:
                                for fan in fan_list:
                                    cumulative_fan[fan]+=formation_multiplier
    return cumulative_fan

In [8]:
def multicolor_dragon_analysis( triplet_collection, duo_collection):
    """
    Fixed first triplet for parallel processing
    """
    preset = {"B1":1, "W2":1, "T3":1, "B4":1, "W5":1, "T6":1, "B7":1, "W8":1, "T9":1 }
    valid_generic_form = 0
    preset_multiplier = [4,6,4,1]
    for fourth_triplet in triplet_collection:
        for duo in duo_collection:
            if test_validity([preset, fourth_triplet, duo]):
                tmp_hand = defaultdict(default_zero)
                # calculate possibilities of forming such hand
                formation_multiplier = 1
                win_tile = None
                for k in preset:
                    tmp_hand[k]+=preset[k]
                for k in fourth_triplet:
                    tmp_hand[k]+=fourth_triplet[k]
                for k in duo:
                    tmp_hand[k]+=duo[k]
                    win_tile = k
                for k in tmp_hand:
                    formation_multiplier*=preset_multiplier[tmp_hand[k]-1]
                valid_generic_form+=formation_multiplier
    valid_generic_form*=6 # 6 combinations with color
    return valid_generic_form

In [9]:
data_path = os.path.join('../fan_analysis_simulated.npy')
with open(data_path, "rb") as f:
    fan_dict = np.load(f, allow_pickle=True).item()
    hash_dict = np.load(f,allow_pickle=True).item()
del fan_dict['全求人']

In [10]:
# 七星不靠
combination_total_bukao7 = 1
# 三色互换
combination_total_bukao7 *= 6
# 16抽14
combination_total_bukao7 *= 9*8/2/1
# 每张牌抽4
combination_total_bukao7 *= 4**14
fan_dict['七星不靠']=combination_total_bukao7
print("{:.3e}".format(combination_total_bukao7))

5.798e+10


In [11]:
# 组合龙 + 全不靠
combination_total_bd = 1
# 三色互换
combination_total_bd *= 6
# 7抽5
combination_total_bd *= 7*6/2/1
# 每张牌抽4
combination_total_bd *= 4**14

print("{:.3e}".format(combination_total_bd))

3.382e+10


In [12]:
# 全不靠
combination_total_bukao = 1
# 三色互换
combination_total_bukao *= 6
# 16抽14
combination_total_bukao *= 16*15/2/1
# 每张牌抽4
combination_total_bukao *= 4**14
combination_total_bukao-=combination_total_bukao7
fan_dict['全不靠']=combination_total_bukao
print("{:.3e}".format(combination_total_bukao))

1.353e+11


In [13]:
# 连七对
combination_total_connected_heptapairs = 1
# 9种选择
combination_total_connected_heptapairs*=9
# 每张抽2
combination_total_connected_heptapairs*=6**7
fan_dict['连七对']=combination_total_connected_heptapairs
print("{:.3e}".format(combination_total_connected_heptapairs))

2.519e+06


In [14]:
# 七对
combination_total_heptapairs = 1
# 34抽7
for i in range(7):
    combination_total_heptapairs*=(34-i)/(7-i)
# 每张抽2
combination_total_heptapairs*=6**7
combination_total_heptapairs-=combination_total_connected_heptapairs
fan_dict['七对']=combination_total_heptapairs
print("{:.3e}".format(combination_total_heptapairs))

1.506e+12


In [15]:
# 组合龙
multicolor_dragon = multicolor_dragon_analysis(triplet_collection,duo_collection)
fan_dict['组合龙']=multicolor_dragon
print("{:.3e}".format(multicolor_dragon))

1.421e+11


In [16]:
# 十三幺
combination_total_thirteen = 1
# *13
combination_total_thirteen*=13
# 12张，4找1
combination_total_thirteen*=4**12
# 一对将
combination_total_thirteen*=6
fan_dict['十三幺']=combination_total_thirteen
print("{:.3e}".format(combination_total_thirteen))

1.309e+09


In [17]:

total_valid_combinations = 0
for hash in hash_dict:
    total_valid_combinations+=hash_dict[hash]
print("Total valid special combination: {:.3e}".format(combination_total_bukao+combination_total_bukao7+combination_total_thirteen+multicolor_dragon+combination_total_heptapairs))
print("Total valid generic combination: {:.3e}".format(total_valid_combinations))
total_valid_combinations+=combination_total_bukao
total_valid_combinations+=combination_total_bukao7
total_valid_combinations+=combination_total_thirteen
total_valid_combinations+=multicolor_dragon
total_valid_combinations+=combination_total_heptapairs
print("Total valid combination: {:.3e}".format(total_valid_combinations))

Total valid special combination: 1.843e+12
Total valid generic combination: 1.278e+13
Total valid combination: 1.462e+13


### 难度拆解
概率导致的难度：组合稀少

行牌导致的难度：进牌困难

假设对手随机打牌：

每多一个可碰刻子（2张等1张），成牌概率*4

每多一个可吃顺子（2张等1张），成牌概率*2

每多一个可吃顺子（2张等2张），成牌概率*4

In [41]:
# 对番数合理性的探究

In [40]:
fan_point_dict = defaultdict(default_list)
fan_point_dict[88]=['大四喜','大三元','九莲宝灯','四杠','连七对','绿一色','十三幺',]
fan_point_dict[64]=['小四喜','小三元','字一色','一色双龙会','清幺九','四暗刻']
fan_point_dict[48]=['一色四同顺','一色四节高']
fan_point_dict[32]=['一色四步高','三杠','混幺九',]
fan_point_dict[24]=['一色三同顺','一色三节高','全双刻','七对','全大','全中','全小','清一色','七星不靠']
fan_point_dict[16]=['清龙','一色三步高','三同刻','三暗刻','三色双龙会','全带五',]
fan_point_dict[12]=['三风刻','大于五','小于五','全不靠','组合龙']
fan_point_dict[8]=['花龙','双暗杠','三色三同顺','三色三节高','推不倒','妙手回春','海底捞月','杠上开花','抢杠和','无番和',]
fan_point_dict[6]=['双箭刻','三色三步高','混一色','五门齐','碰碰和','全求人']
fan_point_dict[4]=['双明杠','全带幺','不求人','和绝张',]
fan_point_dict[2]=['箭刻','圈风刻','门风刻','双同刻','双暗刻','暗杠','断幺','平和','门前清','四归一']
fan_point_dict[1]=['无字','连六','老少副','一般高','幺九刻','明杠','喜相逢','缺一门','边张','嵌张','单钓将','自摸',]

In [41]:

for point in fan_point_dict:
    for fan in fan_point_dict[point]:
        if fan not in fan_dict.keys():
            print(fan)

print('单钓将')
print('门前清')

双暗杠
妙手回春
海底捞月
杠上开花
抢杠和
无番和
全求人
不求人
和绝张
暗杠
自摸
单钓将
门前清


In [42]:
ct_list = []
for fan in fan_dict:
    ct_list.append(fan_dict[fan])
ct_list = list(set(ct_list))
ct_list.sort()
for ct in ct_list:
    for fan in fan_dict:
        if fan_dict[fan]==ct:
            print(fan, ct)

连七对 54
四杠 204
一色四同顺 3906
大四喜 46080
清幺九 53760
绿一色 146082
字一色 161280
三杠 280944
一色三同顺 552960
一色双龙会 839808
一色四节高 898074
小四喜 2250240
九莲宝灯 3538944
全双刻 6082560
混幺九 10429440
大三元 16149120
七对 32277641.999999993
三风刻 62161920
全中 63288972
全大 63288972
全小 66499980
双明杠 67264300
碰碰和 72155088
三同刻 107835520
小三元 265933728
一色三节高 291960644
三色双龙会 301989888
一色四步高 434221056
三色三节高 467614016
推不倒 966549204
全带五 1107503148
十三幺 1308622848
清一色 1357790026
四暗刻 2137006080
大于五 2821022844
小于五 2934113956
明杠 3572282820
双箭刻 6997658400
边张 15486600000
混一色 21515189726
全带幺 34990838176
双同刻 36427658400
四归一 48160596952
七星不靠 57982058496.0
一色三步高 58315262400
圈风刻 63211678904
门风刻 63211678904
三暗刻 67479643440
嵌张 71865495360
五门齐 72738091008
一般高 110526685708
清龙 116813316096
全不靠 135291469824.0
老少副 137817864064
组合龙 142079164416
无字 179517725356
箭刻 181725499432
花龙 203319508992
三色三同顺 217786466304
幺九刻 280475158638
双暗刻 281016674996
断幺 510648277760
缺一门 545854749006
连六 693643990528
三色三步高 824292900864
喜相逢 836638934656
平和 1402249088400
单钓将 17932241478

In [43]:
mj_combination = 0
for fan_type in fan_point_dict[88]:
    mj_combination+=fan_dict[fan_type]
    print("{} 成牌可能性:\t {:.2e}".format(fan_type, fan_dict[fan_type]))
print("{:.3e}".format(mj_combination/len(fan_point_dict[88])))

大四喜 成牌可能性:	 4.61e+04
大三元 成牌可能性:	 1.61e+07
九莲宝灯 成牌可能性:	 3.54e+06
四杠 成牌可能性:	 2.04e+02
连七对 成牌可能性:	 5.40e+01
绿一色 成牌可能性:	 1.46e+05
十三幺 成牌可能性:	 1.31e+09
1.898e+08


In [44]:
mj_combination = 0
for fan_type in fan_point_dict[64]:
    mj_combination+=fan_dict[fan_type]
    print("{} 成牌可能性:\t {:.2e}".format(fan_type, fan_dict[fan_type]))
print("{:.3e}".format(mj_combination/len(fan_point_dict[64])))

小四喜 成牌可能性:	 2.25e+06
小三元 成牌可能性:	 2.66e+08
字一色 成牌可能性:	 1.61e+05
一色双龙会 成牌可能性:	 8.40e+05
清幺九 成牌可能性:	 5.38e+04
四暗刻 成牌可能性:	 2.14e+09
4.010e+08


In [45]:
mj_combination = 0
for fan_type in fan_point_dict[48]:
    mj_combination+=fan_dict[fan_type]
    print("{} 成牌可能性:\t {:.2e}".format(fan_type, fan_dict[fan_type]))
print("{:.3e}".format(mj_combination/len(fan_point_dict[48])))

一色四同顺 成牌可能性:	 3.91e+03
一色四节高 成牌可能性:	 8.98e+05
4.510e+05


In [46]:
mj_combination = 0
for fan_type in fan_point_dict[32]:
    mj_combination+=fan_dict[fan_type]
    print("{} 成牌可能性:\t {:.2e}".format(fan_type, fan_dict[fan_type]))
print("{:.3e}".format(mj_combination/len(fan_point_dict[32])))

一色四步高 成牌可能性:	 4.34e+08
三杠 成牌可能性:	 2.81e+05
混幺九 成牌可能性:	 1.04e+07
1.483e+08


In [47]:
mj_combination = 0
for fan_type in fan_point_dict[24]:
    mj_combination+=fan_dict[fan_type]
    print("{} 成牌可能性:\t {:.2e}".format(fan_type, fan_dict[fan_type]))
print("{:.3e}".format(mj_combination/len(fan_point_dict[24])))

一色三同顺 成牌可能性:	 5.53e+05
一色三节高 成牌可能性:	 2.92e+08
全双刻 成牌可能性:	 6.08e+06
七对 成牌可能性:	 3.23e+07
全大 成牌可能性:	 6.33e+07
全中 成牌可能性:	 6.33e+07
全小 成牌可能性:	 6.65e+07
清一色 成牌可能性:	 1.36e+09
七星不靠 成牌可能性:	 5.80e+10
6.652e+09


In [48]:
mj_combination = 0
for fan_type in fan_point_dict[16]:
    mj_combination+=fan_dict[fan_type]
    print("{} 成牌可能性:\t {:.2e}".format(fan_type, fan_dict[fan_type]))
print("{:.3e}".format(mj_combination/len(fan_point_dict[16])))

清龙 成牌可能性:	 1.17e+11
一色三步高 成牌可能性:	 5.83e+10
三同刻 成牌可能性:	 1.08e+08
三暗刻 成牌可能性:	 6.75e+10
三色双龙会 成牌可能性:	 3.02e+08
全带五 成牌可能性:	 1.11e+09
4.069e+10


In [49]:
mj_combination = 0
for fan_type in fan_point_dict[12]:
    mj_combination+=fan_dict[fan_type]
    print("{} 成牌可能性:\t {:.2e}".format(fan_type, fan_dict[fan_type]))
print("{:.3e}".format(mj_combination/len(fan_point_dict[12])))

三风刻 成牌可能性:	 6.22e+07
大于五 成牌可能性:	 2.82e+09
小于五 成牌可能性:	 2.93e+09
全不靠 成牌可能性:	 1.35e+11
组合龙 成牌可能性:	 1.42e+11
5.664e+10
